# Machine Problem 2
---
By:

*Gonzales, Kyle*

*Gulane, Wineff*

*Mingo, Junry*

*Panchacala, Jean*


In [ ]:
# import Pkg;
# Pkg.add("ForwardDiff");
# Pkg.add("Plots");
# Pkg.add("Symbolics");

### You need to implement the benchmark functions below. You may refer to this page for reference: https://en.wikipedia.org/wiki/Test_functions_for_optimization

In [ ]:
### Benchmark functions (for optimization algorithms)
function beale_func(u)
    x = u[1]
    y = u[2]
    return beale_func(x,y)
end

function beale_func(x,y)
    return (1.5 - x + x*y)^2 + (2.25 - x + x*y^2)^2 + (2.625 - x + x*y^3)^2
end

function booth_func(u)
    x = u[1]
    y = u[2]
    return booth_func(x,y)
end

function booth_func(x, y)
    return (x + 2*y - 7)^2 + (2*x + y - 5)^2
end

function matyas_func(u)
    x = u[1]
    y = u[2]
    return matyas_func(x,y)
end

function matyas_func(x,y)
    return 0.26*(x^2 + y^2) - 0.48*x*y
end

function himmelblau_func(u)
    x = u[1]
    y = u[2]
    return himmelblau_func(x,y)
end

function himmelblau_func(x,y)
    return (x^2 + y - 11)^2 + (x + y^2 - 7)^2
end

function mccormick_func(u)
    x = u[1]
    y = u[2]
    return mccormick_func(x,y)
end

function mccormick_func(x,y)
    return sin(x + y) + (x - y)^2 -1.5*x + 2.5*y + 1
end

function goldstein_price_func(x,y)
    a = 1 + ((x + y + 1)^2)*(19 - 14*x + 3*x^2 - 14*y + 6*x*y + 3*y^2)
    b = 30 + ((2*x - 3*y)^2) * (18 - 32*x + 12*x^2 + 48*y - 36*x*y + 27*y^2)
    return a * b
end

function goldstein_price_func(u)
    x = u[1]
    y = u[2]
    return goldstein_price_func(x,y)
end

function three_hump_camel_func(x,y)
    return 2*x^2 - 1.05*x^4 + (x^6)/6 + x*y + y^2
end

function three_hump_camel_func(u)
    x = u[1]
    y = u[2]
    return three_hump_camel_func(x,y)
end

In [ ]:
using ForwardDiff
using Plots
using Symbolics

### helper functions
function optimize(f, params)
    w = rand(p)
    for t in 1:N
        x = X[t, :]
    end
end

function compute_gradient(f, x)
    grad = ForwardDiff.gradient(f, x)
    return grad
end

function symbolic_2dgradient(f)

    @variables x y

    D_x = Differential(x)
    D_y = Differential(y)

    df_dx = expand_derivatives(D_x(f(x, y)))  # Partial derivative w.r.t. x
    df_dy = expand_derivatives(D_y(f(x, y)))  # Partial derivative w.r.t. y

    println("∂f/∂x: $df_dx")
    println("∂f/∂y: $df_dy")

end

function sgd_momentum(obj_func, num_params, start, γ=0.9, η=0.001, xrange=[-10,10], yrange=[-10,10], levels=200, error=0.00001)
    θ = start
    history = θ
    grad = compute_gradient(obj_func, θ)

    while abs(grad[1]) > error || abs(grad[2]) > error
        v = zeros(num_params)
        grad = compute_gradient(obj_func, θ)
        v = γ .* v + - η .* grad
        θ = θ + v
        history = hcat(history, θ)
    end

    return (round.(θ, digits=3), history)
end

### plotting

function countour_plot(func, x_scatter, y_scatter, xoptimal=[], yoptimal=[], title="Results", xrange=[-10,10], yrange=[-10,10], fill=false, levels=100)
    # used to keep track of how each weight converges. points start on the outside and converge towards the center (minimum point)
    # x_scatter -> x variables of history
    # y_scatter -> y variables of history
    # levels -> number of lines to draw

    # plot benchmark function
    x_vals = range(xrange[1], xrange[2], length = 100)
    y_vals = range(yrange[1], yrange[2], length = 100)

    z_vals = [func(x, y) for y in y_vals, x in x_vals]
    Plots.contour(x_vals, y_vals, z_vals, levels = 50, xlabel = "x", ylabel = "y", fill=fill)

    # Set the axis limits
    margin = 1
    xlims!(-10 - margin, 10 + margin)
    ylims!(-10 - margin, 10 + margin)

    # plot trajectory
    plot!(x_scatter, y_scatter, color=:red, label="Trajectory") # line plot instead of scatter plot
    scatter!([x_scatter[1]], [y_scatter[1]], color=:black,label="Initial Solution")
    scatter!([x_scatter[end]], [y_scatter[end]], markershape=:utriangle, color=:black, label="Best Solution")

    #plot optimal solution
    scatter!(xoptimal, yoptimal, color=:blue, marker=:square, label="Optimal Solution")
    plot!(title=title)
end

function plot_trajectory(func, x_scatter, y_scatter, index=1, show_labels=true)
    colors = [:red, :magenta, :green, :cyan]

    if show_labels
        plot!(x_scatter, y_scatter, color=:red, label="Trajectory") # line plot instead of scatter plot
        scatter!([x_scatter[1]], [y_scatter[1]], color=:black,label="Initial Solution")
        scatter!([x_scatter[end]], [y_scatter[end]], markershape=:utriangle, color=:black, label="Best Solution")
    else
        plot!(x_scatter, y_scatter, color=colors[index], label=false) # line plot instead of scatter plot
        scatter!([x_scatter[1]], [y_scatter[1]], color=:black, label=false)
        scatter!([x_scatter[end]], [y_scatter[end]], markershape=:utriangle, color=:black, label=false)
    end
end

function solve(optimizer, func, starts, xoptimal=[], yoptimal=[], title="Results", xrange=[-10,10], yrange=[-10,10], fill=false, levels=100, )
    x_vals = range(xrange[1], xrange[2], length = 100)
    y_vals = range(yrange[1], yrange[2], length = 100)

    z_vals = [func(x, y) for y in y_vals, x in x_vals]
    Plots.contour(x_vals, y_vals, z_vals, levels = 50, xlabel = "x", ylabel = "y", fill=fill)

    # Set the axis limits
    margin = 1
    xlims!(-10 - margin, 10 + margin)
    ylims!(-10 - margin, 10 + margin)

    for index in 1:length(starts)
        start = starts[index]
        output = optimizer(func, 2, start)
        endpoint = output[1]
        println("Best Point (Start $index): " * string(endpoint))
        history = output[2]

        if index == 1
            plot_trajectory(func, history[1,:], history[2,:])
        else
            plot_trajectory(func, history[1,:], history[2,:], index, false)
        end

    end
    scatter!(xoptimal, yoptimal, color=:blue, marker=:square, label="Optimal Solution")
    plot!(title=title)

end

### helper function to generate random starting values
function random_start(start=-10.0, step=0.01, stop=10.0)
    x = rand(start:step:stop, 2)
    println("Starting Values: " * string(x))
    return x
end

function plot_3d(func, optimizer, points, xoptimals=[], yoptimals=[], title="Results", xrange=[-10,10], yrange=[-10,10], fill=false, levels=100)
    outputs = []
    endpoint = []
    history = []

    for point in points
        output = optimizer(func, 2, point)
        println(output[1])
        endpoint = push!(endpoint, output[1])
        history = push!(history, output[2])
    end
    # Generate x and y data points
    x = range(-10, stop=10, length=50)
    y = range(-10, stop=10, length=50)

    # Create a grid of (x, y) points
    X = repeat(x, 1, length(y))
    Y = repeat(y', length(x), 1)

    # Evaluate the function at each grid point
    Z = func.(X, Y)

    # Create 3D surface plot
    surface(x, y, Z, xlabel="X", ylabel="Y", zlabel="Z", title="3D Function Plot")

    # Adding a line
    for i in 1:length(points)
        x_line = history[i][1,:]
        y_line = history[i][2,:]
        z_line = func.(x_line, y_line)
        plot!(x_line, y_line, z_line, color="red", linewidth=2, label="Line", primary=false)
        scatter!([points[i][1]], [points[i][2]], [func(points[i][1], points[i][2])], label="Starting point", marker=:circle, primary=false)
        scatter!([endpoint[i][1]], [endpoint[i][2]], [func(endpoint[i][1], endpoint[i][2])], markershape=:utriangle ,color=:green, label="Endpoint",primary=false)
    end

    z_optimals = []
    for i in 1:length(xoptimal)
        push!(z_optimals, func(xoptimal[i], yoptimal[i]))
    end
    scatter!(xoptimal, yoptimal, z_optimals, color=:blue, marker=:square, label="Optimal Solution")
end

In [ ]:
start_1 = [10,-10];
start_2 = [-4, 4];
start_3 = [0, 0];
start_4 = [5,5];

starts = [start_1, start_2, start_3, start_4]

## RMSProp

In [ ]:
function rmsprop(obj_func, num_params, start, γ=0.9, η=0.001, xrange=[-10,10], yrange=[-10,10], levels=200, error=0.00001)

    θ = start
    history = θ
    # grad = compute_gradient(obj_func, θ)
    epochs = 0
    v = zeros(num_params)

    while true
        grad = compute_gradient(obj_func, θ)
        if !(abs(grad[1]) > error || abs(grad[2]) > error) || epochs > 30000
            break
        end
        v = γ .* v + (1-γ) .* grad .^ 2
        θ = θ - η .* grad ./ sqrt.(v .+ 0.0000001)
        history = hcat(history, θ)
        epochs += 1
    end
    println(epochs)
    return (round.(θ, digits=3), history)
end
# ref: https://github.com/keras-team/keras/blob/v3.3.3/keras/src/optimizers/rmsprop.py#L6

Task 2. For each function, determine the trajectory of the algorithm given the following starting points

### Beale Function

In [ ]:
func = beale_func
title = "RMSProp | Beale Function"
xoptimal = [3.0]
yoptimal = [0.5]

In [ ]:
solve(rmsprop, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, rmsprop, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)

In [ ]:
solve(rmsprop, func, random_starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, rmsprop, random_starts, xoptimal, yoptimal)

### Booth Function

In [ ]:
func = booth_func
title = "RMSProp | Booth Function"
xoptimal = [1.0]
yoptimal = [3.0]

In [ ]:
solve(rmsprop, func, starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, rmsprop, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)

In [ ]:
solve(rmsprop, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, rmsprop, random_starts, xoptimal, yoptimal)

### Matyas Function

In [ ]:
func = matyas_func
title = "RMSProp | Matyas Function"
xoptimal = [0]
yoptimal = [0]

In [ ]:
solve(rmsprop, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, rmsprop, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(rmsprop, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, rmsprop, random_starts, xoptimal, yoptimal)

### Himmelblau Function

In [ ]:
func = himmelblau_func
title = "RMSProp | HimmelBlau Function"
xoptimal = [3, -2.805118, -3.779310, 3.584428]
yoptimal = [2, 3.131312, -3.283186, -1.848126]

In [ ]:
solve(rmsprop, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, rmsprop, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(rmsprop, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, rmsprop, random_starts, xoptimal, yoptimal)

### Mccormick Function

In [ ]:
func = mccormick_func
title = "RMSProp | Mccormick Function"
xoptimal = [-0.54719]
yoptimal = [-1.54719]

In [ ]:
solve(rmsprop, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, rmsprop, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(rmsprop, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, rmsprop, random_starts, xoptimal, yoptimal)

### Goldstein-Price Function

In [ ]:
func = goldstein_price_func
title = "RMSProp | Goldstein-Price Function"
xoptimal = [0]
yoptimal = [-1]

In [ ]:
solve(rmsprop, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, rmsprop, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(rmsprop, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, rmsprop, random_starts, xoptimal, yoptimal)

### Three-Hump Camel Function

In [ ]:
func = three_hump_camel_func
title = "RMSProp | Three-Hump Camel Function"
xoptimal = [0]
yoptimal = [0]

In [ ]:
solve(rmsprop, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, rmsprop, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(rmsprop, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, rmsprop, random_starts, xoptimal, yoptimal)

## Adaptive Moment Estimation (Adam)

In [ ]:
### Implement the Adam here
### learning algorithms
function adam(obj_func, num_params, start, γ=0.9, η=0.001, levels=200, error=0.00001, β1 = 0.9, β2 = 0.999)

    θ = start
    history = θ
    grad = compute_gradient(obj_func, θ)
    epochs = 0
    v = zeros(num_params)
    m = zeros(num_params)
    while abs(grad[1]) > error || abs(grad[2]) > error
        # update this code block with Adam
        if epochs > 50000
            break
        end

        m = β1 * m + (1 - β1) * grad
        m̂ = m / (1- β1^(epochs+1))

        v = β2 * v + (1 - β2) * grad .^ 2
        v̂ = v / (1 - β2^(epochs+1))

        θ = θ - η .* m̂ ./ (sqrt.(v̂) .+ 1e-8)  # Adjusted learning rate

        grad = compute_gradient(obj_func, θ)
        history = hcat(history, θ)
        epochs += 1
    end
    """
        instead of updating the weight with the current gradient, consider the previous gradient
    """
    println(epochs)
    return (round.(θ, digits=3), history)
end

In [ ]:
start_1 = [10,-10];
start_2 = [-4, 4];
start_3 = [0, 0];
start_4 = [5,5];

starts = [start_1, start_2, start_3, start_4]

### Beale Function

In [ ]:
func = beale_func
title = "Adam | Beale Function"
xoptimal = [3.0]
yoptimal = [0.5]

In [ ]:
solve(adam, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, adam, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(adam, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, adam, random_starts, xoptimal, yoptimal)

### Booth Function

In [ ]:
func = booth_func
title = "Adam | Booth Function"
xoptimal = [1.0]
yoptimal = [3.0]

In [ ]:
solve(adam, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, adam, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(adam, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, adam, random_starts, xoptimal, yoptimal)

### Matyas Function

In [ ]:
func = matyas_func
title = "Adam | Matyas Function"
xoptimal = [0]
yoptimal = [0]

In [ ]:
solve(adam, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, adam, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(adam, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, adam, random_starts, xoptimal, yoptimal)

### Himmelblau Function

In [ ]:
func = himmelblau_func
title = "Adam | HimmelBlau Function"
xoptimal = [3, -2.805118, -3.779310, 3.584428]
yoptimal = [2, 3.131312, -3.283186, -1.848126]

In [ ]:
solve(adam, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, adam, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(adam, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, adam, random_starts, xoptimal, yoptimal)

### Mccormick Function

In [ ]:
func = mccormick_func
title = "Adam | Mccormick Function"
xoptimal = [-0.54719]
yoptimal = [-1.54719]

In [ ]:
solve(adam, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, adam, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(adam, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, adam, random_starts, xoptimal, yoptimal)

### Goldstein-Price Function

In [ ]:
func = goldstein_price_func
title = "Adam | Goldstein-Price Function"
xoptimal = [0]
yoptimal = [-1]

In [ ]:
solve(adam, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, adam, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(adam, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, adam, random_starts, xoptimal, yoptimal)

### Three-Hump Camel Function

In [ ]:
func = three_hump_camel_func
title = "Adam | Three-Hump Camel Function"
xoptimal = [0]
yoptimal = [0]

In [ ]:
solve(adam, func, starts, xoptimal, yoptimal, title)

In [ ]:
plot_3d(func, adam, starts, xoptimal, yoptimal)

In [ ]:
random_starts = []
for i in 1:4
    push!(random_starts,random_start())
end
print(random_starts)
solve(adam, func, random_starts, xoptimal, yoptimal,title)

In [ ]:
plot_3d(func, adam, random_starts, xoptimal, yoptimal)


objective: try to let the learning algorithms reach the optimal points of these benchmark functions

Strategy
1. define the benchmark function
2. define the starting point (given by sir)
3. use learning algorithm, given the benchmark function, starting point
4. see if the starting point converges to one of the global minimum points (refer to wiki)

Tasks
1. implement benchmark functions
2. implement learning algorithms
3. determine the trajectory of each function using the given starting points (use contour_plot)
4. determine the trajectory of each function using randomly generated starting points ()

Tips
- be careful for points that vanish - explode
- beale function is very difficult - you may need to change learning rate and v to help the learning rate converge
- be careful for when you get stuck at local minimums - gradient is 0, but there is a deeper trough based on the overall landscape
  - this tells us that the function is stuck at a local minimum - the area has a lot of local minimums
  - tune learning rate, change optimization functions

question